<a href="https://colab.research.google.com/github/omidr1348/news_analysis/blob/main/news_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omidrahmanzadeh","key":"773ade4c3f6f4ef787e5e5d05213a6c3"}'}

In [ ]:
import os

# Make directory for Kaggle API key
os.makedirs('/root/.kaggle', exist_ok=True)

# Move kaggle.json to the proper location
!mv kaggle.json /root/.kaggle/kaggle.json

# Set correct permissions
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!pip install -q gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have nu

In [ ]:
!pip install -q kaggle

In [ ]:
# Download the dataset using the Kaggle API
!kaggle datasets download clmentbisaillon/fake-and-real-news-dataset

Dataset URL: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
License(s): CC-BY-NC-SA-4.0
  0% 0.00/41.0M [00:00<?, ?B/s]
100% 41.0M/41.0M [00:00<00:00, 1.17GB/s]


In [ ]:
# Unzip the downloaded file
!unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

The dataset is downloaded as a zip file. We need to unzip it to access the CSV files.

Now that the files are unzipped, we can load the data into pandas DataFrames. There are two CSV files: `Fake.csv` and `True.csv`.

In [ ]:
import pandas as pd

# Load the fake news data
fake_df = pd.read_csv('Fake.csv')

# Load the real news data
true_df = pd.read_csv('True.csv')

# Display the first few rows of each DataFrame
print("Fake News Data:")
display(fake_df.head())

print("\nReal News Data:")
display(true_df.head())

Fake News Data:


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"



Real News Data:


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import os

# Set NLTK data path explicitly
nltk_data_path = '/content/nltk_data'
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.append(nltk_data_path)


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Add a 'label' column to distinguish between fake and real news
fake_df['label'] = 1  # Label for fake news
true_df['label'] = 0  # Label for real news

# Combine the two dataframes
news_df = pd.concat([fake_df, true_df], ignore_index=True)

# Display the first few rows of the combined dataframe
print("Combined News Data:")
display(news_df.head())
display(news_df.tail())

Combined News Data:


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


,title,text,subject,date,label
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0
44897,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",0


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
# Download necessary NLTK data
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

try:
    word_tokenize('test') # Attempt to tokenize to check for punkt
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt_tab/')
except LookupError:
    nltk.download('punkt_tab')


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]


    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Join tokens back into a string
    return ' '.join(lemmatized)

# Apply the preprocessing function to the 'text' column
news_df['cleaned_text'] = news_df['text'].apply(preprocess_text)

# Display the first few rows with the new 'cleaned_text' column
print("News Data with Cleaned Text:")
display(news_df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


News Data with Cleaned Text:


,title,text,subject,date,label,cleaned_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1,donald trump wish american happy new year leav...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1,house intelligence committee chairman devin nu...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1,friday revealed former milwaukee sheriff david...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1,christmas day donald trump announced would bac...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1,pope francis used annual christmas day message...


In [ ]:
news_df['cleaned_text'][1]

'house intelligence committee chairman devin nunes going bad day assumption like many u christopher steeledossier prompted russia investigation lashing department justice fbi order protect trump happens dossier started investigation according document obtained new york timesformer trump campaign adviser george papadopoulos drunk wine bar revealed knowledge russian opposition research hillary clintonon top papadopoulos covfefe boy trump administration alleged much larger role none damning drunken fool wine bar coffee boy help arrange new york meeting trump president abdel fattah elsisi egypt two month election known former aide set meeting world leader trump team trump ran merely coffee boyin may 2016 papadopoulos revealed australian diplomat alexander downer russian official shopping around possible dirt thendemocratic presidential nominee hillary clinton exactly much mr papadopoulos said night kensington wine room australian alexander downer unclear report state two month later leaked

## Feature engineering




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer
# Consider limiting max_features to manage the size of the feature matrix
# Consider using ngram_range to capture multi-word phrases
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Apply TF-IDF to the 'cleaned_text' column
tfidf_features = tfidf_vectorizer.fit_transform(news_df['cleaned_text'])

# Print the shape of the resulting feature matrix
print("Shape of TF-IDF features:", tfidf_features.shape)

Shape of TF-IDF features: (44898, 5000)


## Data splitting




In [ ]:
from sklearn.model_selection import train_test_split

# Separate features (TF-IDF matrix) and target variable (label)
X = tfidf_features
y = news_df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Print the shapes of the resulting sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (33673, 5000)
Shape of X_test: (11225, 5000)
Shape of y_train: (33673,)
Shape of y_test: (11225,)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
model = LogisticRegression(max_iter=1000)

# The model is now chosen and ready for training in the next step.
# We will not train it in this step, just select it.
print("Selected Model: Logistic Regression")

Selected Model: Logistic Regression


In [ ]:
# Fit the model to the training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

## Model evaluation


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9882
Precision: 0.9900
Recall: 0.9874
F1-score: 0.9887


## Prediction




In [ ]:
# Define new text data
new_texts = [
    "This is a sample real news article about politics.",
    "Breaking: President makes outrageous claim about election results!",
    "Scientists discover new planet capable of supporting life.",
    "URGENT: Secret documents reveal shocking truth about government conspiracy!"
]

# Preprocess the new text data
cleaned_new_texts = [preprocess_text(text) for text in new_texts]

# Transform the preprocessed new text data into TF-IDF features
new_tfidf_features = tfidf_vectorizer.transform(cleaned_new_texts)

# Predict labels for the new TF-IDF features
predictions = model.predict(new_tfidf_features)

# Print the original new text and its corresponding predicted label
for i, text in enumerate(new_texts):
    predicted_label = "Fake" if predictions[i] == 1 else "Real"
    print(f"Original Text: {text}\nPredicted Label: {predicted_label}\n")

Original Text: This is a sample real news article about politics.
Predicted Label: Fake

Original Text: Breaking: President makes outrageous claim about election results!
Predicted Label: Fake

Original Text: Scientists discover new planet capable of supporting life.
Predicted Label: Fake

Original Text: URGENT: Secret documents reveal shocking truth about government conspiracy!
Predicted Label: Fake



In [ ]:
def tokenize(text):
    words = word_tokenize(text)  # tokenize
    return words

In [ ]:
sentences = news_df['cleaned_text'].apply(tokenize).tolist()

In [ ]:
import random

# Take a random sample of 1000 rows from the 'sentences' list
sample_size = 10000
if len(sentences) >= sample_size:
    sample_sentences = random.sample(sentences, sample_size)
else:
    sample_sentences = sentences
    print(f"Warning: Not enough sentences to take a sample of {sample_size}. Using all {len(sentences)} sentences.")


# Print the number of sentences in the sample
print(f"Number of sentences in the sample: {len(sample_sentences)}")

# Display the first few sample sentences
print("\nFirst 5 sample sentences:")
for i in range(min(5, len(sample_sentences))):
    print(f"Sentence {i+1}: {sample_sentences[i]}")

Number of sentences in the sample: 10000

First 5 sample sentences:
Sentence 1: ['noticed', 'trend', 'across', 'america', 'disturbing', 'community', 'muslim', 'assimilate', 'want', 'american', 'bend', 'every', 'whim', 'noticed', 'last', 'election', 'nonenglish', 'speaking', 'forum', 'held', 'minneapolis', '2016', 'electionilhan', 'omar', 'addressing', 'room', 'english', 'far', 'candidate', 'speech', 'relay', 'said', 'pictwittercomoh3jtph91i', 'eric', 'roper', 'stribroper', 'march', '2', '2016no', 'english', 'spoken', 'call', 'assimilationquite', 'scene', 'omar', 'left', 'noor', 'right', 'supporter', 'break', 'pictwittercomo2arz18hkc', 'eric', 'roper', 'stribroper', 'march', '2', '2016an', 'uptick', 'muslim', 'activist', 'running', 'position', 'public', 'office', 'noticeable', 'wanted', 'find', 'behind', 'new', 'american', 'leader', 'project', 'recruit', 'anyone', 'white', 'run', 'officenew', 'group', 'maryland', 'recruiting', 'muslim', 'candidate', 'run', 'officecandidates', 'across', 

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.0-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.0-py3-none-any.whl (292 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4487346 sha256=55cd3e3c40bcfde418668fc567ff5026903cd8408f5876a063ce3fa956ffe8c2
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext


In [ ]:
import fasttext
import numpy as np
import os

# Save the sampled sentences to a temporary file
temp_file_path = "sample_sentences.txt"
with open(temp_file_path, "w", encoding="utf-8") as f:
    for sentence in sample_sentences:
        f.write(" ".join(sentence) + "\n")


# Train a FastText model on the sampled sentences
# The parameters epoch and lr were set during initialization in the previous attempt
model = fasttext.train_unsupervised(temp_file_path, dim=100, epoch=2, lr=0.1)

# Remove the temporary file
os.remove(temp_file_path)


def get_embedding(text):
    # fasttext model.get_sentence_vector converts text to a vector
    return model.get_sentence_vector(text)

# Generate embeddings for the cleaned text data
X = np.array([get_embedding(text) for text in news_df['cleaned_text']])
y = news_df['label']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# آموزش مدل Logistic Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)

# پیش‌بینی
y_pred = clf.predict(X_test)

# ارزیابی
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9684484038604306


In [ ]:
!pip install -q transformers